In [11]:
try:
    import pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
except ImportError:
    import sys
    !{sys.executable} -m pip install pytesseract
    import pytesseract
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

try:
    from PIL import Image
except ImportError:
    import sys
    !{sys.executable} -m pip install pillow
    from PIL import Image

try:
    import pandas as pd
except ImportError:
    import sys
    !{sys.executable} -m pip install pandas
    import pandas as pd

try:
    import re
except ImportError:
    raise ImportError("The 're' module is part of the Python standard library and should always be available.")

try:
    from datetime import datetime
except ImportError:
    raise ImportError("The 'datetime' module is part of the Python standard library and should always be available.")

import logging

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
class OCRProcessor:
    """OCR processing with emphasis on preprocessing quality over engine selection"""
    
    def __init__(self):
        self.confidence_threshold = 70  # Minimum confidence for acceptance
        
    def preprocess_image(self, image_path):
        """
        Critical preprocessing pipeline based on practitioner consensus
        Priority: CLAHE -> Adaptive threshold -> Noise reduction -> ROI
        """
        logger.info(f"Processing {image_path}")
        
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 1. CLAHE for contrast enhancement (most important step)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(gray)
        
        # 2. Adaptive thresholding (better than global for financial tables)
        thresh = cv2.adaptiveThreshold(
            enhanced, 255, cv2.THRESH_BINARY, cv2.THRESH_BINARY, 11, 2
        )
        
        # 3. Noise reduction via median filtering
        denoised = cv2.medianBlur(thresh, 3)
        
        # 4. Morphological operations to clean up table structure
        kernel = np.ones((2,2), np.uint8)
        cleaned = cv2.morphologyEx(denoised, cv2.MORPH_CLOSE, kernel)
        
        return cleaned
    
    def extract_text_with_confidence(self, processed_img):
        """Extract text with confidence scores for validation"""
        
        # Get detailed OCR data including confidence
        custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789./- ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
        
        # Extract text with confidence
        ocr_data = pytesseract.image_to_data(
            processed_img, 
            output_type=pytesseract.Output.DICT,
            config=custom_config
        )
        
        return ocr_data
    
    def validate_financial_number(self, text):
        """Business logic validation for financial numbers"""
        # Pattern for financial numbers: optional parentheses, digits, commas, decimal
        pattern = r'^[\(]?[\d,]+\.?\d*[\)]?$'
        
        if re.match(pattern, text.strip()):
            # Clean and convert
            cleaned = text.replace(',', '').replace('(', '-').replace(')', '')
            try:
                return float(cleaned)
            except ValueError:
                return None
        return None
    
    def extract_data_from_image(self, image_path):
        """Main extraction method with comprehensive validation"""
        
        processed_img = self.preprocess_image(image_path)
        ocr_data = self.extract_text_with_confidence(processed_img)
        
        # Extract high-confidence text
        extracted_data = []
        for i, confidence in enumerate(ocr_data['conf']):
            if int(confidence) > self.confidence_threshold:
                text = ocr_data['text'][i].strip()
                if text:
                    extracted_data.append({
                        'text': text,
                        'confidence': confidence,
                        'x': ocr_data['left'][i],
                        'y': ocr_data['top'][i]
                    })
        
        logger.info(f"Extracted {len(extracted_data)} high-confidence text elements")
        return extracted_data, processed_img

In [4]:
class DataParser:
    """Parse OCR results into structured financial data"""
    
    def __init__(self):
        self.date_patterns = [
            r'\d{2}/\d{2}/\d{2}',  # MM/DD/YY
            r'\d{2}/\d{2}/\d{4}',  # MM/DD/YYYY
        ]
    
    def parse_date(self, text):
        """Parse various date formats"""
        for pattern in self.date_patterns:
            if re.match(pattern, text):
                try:
                    # Try MM/DD/YY format first (common in financial data)
                    if len(text.split('/')[2]) == 2:
                        date_obj = datetime.strptime(text, '%m/%d/%y')
                    else:
                        date_obj = datetime.strptime(text, '%m/%d/%Y')
                    return date_obj.strftime('%Y-%m-%d')
                except ValueError:
                    continue
        return None
    
    def extract_table_data(self, extracted_data, data_type="bond"):
        """Extract structured data from OCR results"""
        
        # Sort by vertical position to process row by row
        sorted_data = sorted(extracted_data, key=lambda x: (x['y'], x['x']))
        
        parsed_rows = []
        current_row = []
        current_y = None
        y_tolerance = 5  # Pixels tolerance for same row
        
        for item in sorted_data:
            if current_y is None or abs(item['y'] - current_y) <= y_tolerance:
                current_row.append(item)
                current_y = item['y'] if current_y is None else current_y
            else:
                if current_row:
                    parsed_rows.append(self.process_row(current_row, data_type))
                current_row = [item]
                current_y = item['y']
        
        # Don't forget the last row
        if current_row:
            parsed_rows.append(self.process_row(current_row, data_type))
        
        # Filter out None rows and create DataFrame
        valid_rows = [row for row in parsed_rows if row is not None]
        
        if data_type == "bond":
            columns = ['date', 'bond_price', 'price_change']
        else:  # futures
            columns = ['date', 'future_price', 'bid_price']
            
        df = pd.DataFrame(valid_rows, columns=columns)
        return df.dropna(subset=['date'])  # Remove rows without valid dates
    
    def process_row(self, row_items, data_type):
        """Process a single row of OCR data"""
        
        # Sort items in row by x-coordinate (left to right)
        row_items.sort(key=lambda x: x['x'])
        
        date_val = None
        price_val = None
        secondary_val = None
        
        ocr_processor = OCRProcessor()
        
        for item in row_items:
            text = item['text']
            
            # Try to parse as date
            parsed_date = self.parse_date(text)
            if parsed_date and not date_val:
                date_val = parsed_date
                continue
            
            # Try to parse as financial number
            parsed_number = ocr_processor.validate_financial_number(text)
            if parsed_number is not None:
                if price_val is None:
                    price_val = parsed_number
                elif secondary_val is None:
                    secondary_val = parsed_number
        
        if date_val and price_val is not None:
            return [date_val, price_val, secondary_val]
        
        return None

In [5]:
class StressAnalyzer:
    """Stress scenario analysis following industry best practices"""
    
    def __init__(self):
        self.business_days_mapping = {3: '3BD', 7: '7BD', 14: '14BD'}
    
    def calculate_stress(self, df, price_col, days=3):
        """
        Calculate nBD stress using formula:
        stress = (price_today / price_today_minus_nBD) - 1
        """
        logger.info(f"Calculating {days}BD stress for {price_col}")
        
        # Ensure data is sorted by date
        df_sorted = df.sort_values('date').copy()
        df_sorted['date'] = pd.to_datetime(df_sorted['date'])
        
        # Calculate lagged prices (n business days ago)
        df_sorted[f'{price_col}_lag_{days}'] = df_sorted[price_col].shift(days)
        
        # Calculate stress
        stress_col = f'{price_col}_stress_{days}BD'
        df_sorted[stress_col] = (df_sorted[price_col] / df_sorted[f'{price_col}_lag_{days}']) - 1
        
        return df_sorted.dropna(subset=[stress_col])
    
    def linear_regression_analysis(self, bond_stress, future_stress, days=3):
        """Linear regression with comprehensive validation"""
        
        # Prepare data
        X = bond_stress.values.reshape(-1, 1)
        y = future_stress.values
        
        # Fit linear model
        model = LinearRegression()
        model.fit(X, y)
        
        # Predictions and metrics
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)
        
        # Calculate additional metrics
        residuals = y - y_pred
        mse = np.mean(residuals**2)
        
        results = {
            'days': days,
            'alpha': model.intercept_,
            'beta': model.coef_[0],
            'r_squared': r2,
            'mse': mse,
            'n_observations': len(y),
            'model': model
        }
        
        logger.info(f"{days}BD Model: α={results['alpha']:.4f}, β={results['beta']:.4f}, R²={r2:.4f}")
        
        return results
    
    def create_stress_plots(self, combined_df, periods=[3, 7, 14]):
        """Generate comprehensive stress analysis plots"""
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Bond-Futures Stress Scenario Analysis', fontsize=16)
        
        colors = ['blue', 'red', 'green']
        results_summary = []
        
        # Plot 1: Multiple period comparison
        ax1 = axes[0, 0]
        for i, days in enumerate(periods):
            if f'bond_price_stress_{days}BD' in combined_df.columns and f'future_price_stress_{days}BD' in combined_df.columns:
                bond_stress = combined_df[f'bond_price_stress_{days}BD'].dropna()
                future_stress = combined_df[f'future_price_stress_{days}BD'].dropna()
                
                if len(bond_stress) > 0 and len(future_stress) > 0:
                    # Align data
                    min_len = min(len(bond_stress), len(future_stress))
                    bond_stress = bond_stress.iloc[-min_len:]
                    future_stress = future_stress.iloc[-min_len:]
                    
                    ax1.scatter(bond_stress, future_stress, alpha=0.6, 
                              color=colors[i], label=f'{days}BD', s=30)
                    
                    # Fit line
                    if len(bond_stress) > 1:
                        results = self.linear_regression_analysis(bond_stress, future_stress, days)
                        results_summary.append(results)
                        
                        x_range = np.linspace(bond_stress.min(), bond_stress.max(), 100)
                        y_line = results['alpha'] + results['beta'] * x_range
                        ax1.plot(x_range, y_line, color=colors[i], linestyle='--', linewidth=2)
        
        ax1.set_xlabel('Bond Index Stress')
        ax1.set_ylabel('Future Stress')
        ax1.set_title('Stress Relationships by Time Period')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: 3BD detailed analysis (main focus)
        ax2 = axes[0, 1]
        if results_summary:
            main_result = results_summary[0]  # 3BD typically first
            bond_stress = combined_df['bond_price_stress_3BD'].dropna()
            future_stress = combined_df['future_price_stress_3BD'].dropna()
            
            min_len = min(len(bond_stress), len(future_stress))
            if min_len > 0:
                bond_stress = bond_stress.iloc[-min_len:]
                future_stress = future_stress.iloc[-min_len:]
                
                ax2.scatter(bond_stress, future_stress, alpha=0.7, color='darkblue', s=40)
                
                x_range = np.linspace(bond_stress.min(), bond_stress.max(), 100)
                y_line = main_result['alpha'] + main_result['beta'] * x_range
                ax2.plot(x_range, y_line, color='red', linewidth=2, 
                        label=f'y = {main_result["alpha"]:.4f} + {main_result["beta"]:.4f}x')
                
                ax2.set_xlabel('Bond Index 3BD Stress')
                ax2.set_ylabel('Future 3BD Stress')
                ax2.set_title(f'3BD Linear Relationship (R² = {main_result["r_squared"]:.4f})')
                ax2.legend()
                ax2.grid(True, alpha=0.3)
        
        # Plot 3: Time series of stress values
        ax3 = axes[1, 0]
        if 'date' in combined_df.columns:
            plot_df = combined_df.set_index('date')
            for i, days in enumerate(periods):
                bond_col = f'bond_price_stress_{days}BD'
                future_col = f'future_price_stress_{days}BD'
                
                if bond_col in plot_df.columns:
                    ax3.plot(plot_df.index, plot_df[bond_col], 
                            color=colors[i], alpha=0.7, label=f'Bond {days}BD')
                if future_col in plot_df.columns:
                    ax3.plot(plot_df.index, plot_df[future_col], 
                            color=colors[i], alpha=0.7, linestyle='--', label=f'Future {days}BD')
        
        ax3.set_xlabel('Date')
        ax3.set_ylabel('Stress Value')
        ax3.set_title('Time Series of Stress Values')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax3.tick_params(axis='x', rotation=45)
        
        # Plot 4: Model diagnostics
        ax4 = axes[1, 1]
        if results_summary:
            metrics = ['R²', 'Beta', 'Alpha']
            period_labels = [f'{r["days"]}BD' for r in results_summary]
            
            r2_values = [r['r_squared'] for r in results_summary]
            beta_values = [r['beta'] for r in results_summary]
            alpha_values = [abs(r['alpha']) for r in results_summary]  # Absolute for better visualization
            
            x = np.arange(len(period_labels))
            width = 0.25
            
            ax4.bar(x - width, r2_values, width, label='R²', alpha=0.7)
            ax4.bar(x, beta_values, width, label='Beta', alpha=0.7)
            ax4.bar(x + width, alpha_values, width, label='|Alpha|', alpha=0.7)
            
            ax4.set_xlabel('Time Period')
            ax4.set_ylabel('Metric Value')
            ax4.set_title('Model Diagnostics Comparison')
            ax4.set_xticks(x)
            ax4.set_xticklabels(period_labels)
            ax4.legend()
            ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('stress_analysis_results.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        return results_summary

In [7]:
def main():
    """Main execution pipeline integrating OCR and stress analysis"""
    
    logger.info("Starting integrated OCR and stress modeling pipeline")
    
    # Initialize processors
    ocr_processor = OCRProcessor()
    data_parser = DataParser()
    stress_analyzer = StressAnalyzer()
    
    # Phase 1: OCR Data Extraction
    logger.info("=== Phase 1: OCR Data Extraction ===")
    
    try:
        # Extract bond index data (BEHLTREU)
        logger.info("Processing BEHLTREU bond index data...")
        bond_data, bond_img = ocr_processor.extract_data_from_image('../Entry_Technical_Test/behltreu.png')
        bond_df = data_parser.extract_table_data(bond_data, data_type="bond")
        logger.info(f"Extracted {len(bond_df)} bond price records")
        
        # Extract futures data (AHWM5)
        logger.info("Processing AHWM5 futures data...")
        futures_data, futures_img = ocr_processor.extract_data_from_image('../Entry_Technical_Test/ahwm5.png')
        futures_df = data_parser.extract_table_data(futures_data, data_type="future")
        logger.info(f"Extracted {len(futures_df)} futures price records")
        
        # Merge datasets on date
        combined_df = pd.merge(bond_df[['date', 'bond_price']], 
                              futures_df[['date', 'future_price']], 
                              on='date', how='inner')
        
        logger.info(f"Combined dataset: {len(combined_df)} overlapping records")
        
        # Save raw data
        combined_df.to_csv('extracted_data.csv', index=False)
        logger.info("Raw data saved to extracted_data.csv")
        
    except Exception as e:
        logger.error(f"OCR extraction failed: {e}")
        # Fallback: create sample data for demonstration
        logger.warning("Creating sample data for demonstration...")
        dates = pd.date_range(start='2025-04-01', end='2025-07-01', freq='B')
        np.random.seed(42)
        
        bond_prices = 300 + np.cumsum(np.random.normal(0, 0.5, len(dates)))
        future_prices = bond_prices + np.random.normal(0, 0.2, len(dates))
        
        combined_df = pd.DataFrame({
            'date': dates.strftime('%Y-%m-%d'),
            'bond_price': bond_prices,
            'future_price': future_prices
        })
        
        logger.info(f"Created {len(combined_df)} sample records")
    
    # Phase 2: Stress Scenario Analysis  
    logger.info("=== Phase 2: Stress Scenario Analysis ===")
    
    if len(combined_df) < 15:  # Need minimum data for meaningful analysis
        logger.warning("Insufficient data for stress analysis")
        return
    
    # Calculate stress for multiple periods
    periods = [3, 7, 14]
    for days in periods:
        if len(combined_df) > days:
            # Calculate bond stress
            bond_stress_df = stress_analyzer.calculate_stress(
                combined_df, 'bond_price', days=days
            )
            combined_df = pd.merge(combined_df, 
                                 bond_stress_df[['date', f'bond_price_stress_{days}BD']], 
                                 on='date', how='left')
            
            # Calculate futures stress
            future_stress_df = stress_analyzer.calculate_stress(
                combined_df, 'future_price', days=days
            )
            combined_df = pd.merge(combined_df, 
                                 future_stress_df[['date', f'future_price_stress_{days}BD']], 
                                 on='date', how='left')
    
    # Generate comprehensive analysis
    results_summary = stress_analyzer.create_stress_plots(combined_df, periods)
    
    # Save final results
    combined_df.to_csv('stress_analysis_data.csv', index=False)
    
    # Print summary statistics
    logger.info("=== Analysis Summary ===")
    for result in results_summary:
        print(f"\n{result['days']}BD Linear Model Results:")
        print(f"  Formula: Future_Stress = {result['alpha']:.4f} + {result['beta']:.4f} × Bond_Stress")
        print(f"  R-squared: {result['r_squared']:.4f}")
        print(f"  Observations: {result['n_observations']}")
    
    print(f"\nFinal dataset shape: {combined_df.shape}")
    print("Files generated:")
    print("  - extracted_data.csv (raw OCR data)")  
    print("  - stress_analysis_data.csv (with stress calculations)")
    print("  - stress_analysis_results.png (comprehensive plots)")
    
    logger.info("Pipeline completed successfully")

In [12]:
main()

2025-07-22 14:19:31,963 - INFO - Starting integrated OCR and stress modeling pipeline
2025-07-22 14:19:31,966 - INFO - === Phase 1: OCR Data Extraction ===
2025-07-22 14:19:31,969 - INFO - Processing BEHLTREU bond index data...
2025-07-22 14:19:31,969 - INFO - Processing ../Entry_Technical_Test/behltreu.png
2025-07-22 14:19:33,848 - INFO - Extracted 3 high-confidence text elements
2025-07-22 14:19:33,852 - INFO - Extracted 0 bond price records
2025-07-22 14:19:33,852 - INFO - Processing AHWM5 futures data...
2025-07-22 14:19:33,852 - INFO - Processing ../Entry_Technical_Test/ahwm5.png
2025-07-22 14:19:34,770 - INFO - Extracted 11 high-confidence text elements
2025-07-22 14:19:34,781 - INFO - Extracted 0 futures price records
2025-07-22 14:19:34,781 - INFO - Combined dataset: 0 overlapping records
2025-07-22 14:19:34,812 - INFO - Raw data saved to extracted_data.csv
2025-07-22 14:19:34,814 - INFO - === Phase 2: Stress Scenario Analysis ===
2025-07-22 14:19:34,814 - WARNING - Insufficien